# Data Analysis of COVID 2019

## Summary 
---
* Objective：Compare the patterns of SARS 2003 with current COVID 2019
* Methods：Time Series Analysis
* Results：Under way

## Experiment
---
* 
* 

## Results
---
* 
* 

### Topic
* item
* item 

### Topic
* item 
* item 

## Future Works
---

## Reference 
---
* item 
* item 

# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import pandas as pd 

from pandas_datareader import DataReader
from datetime import datetime
%matplotlib inline
from scipy.stats import norm
import pandas_datareader.data as web

import statsmodels.api as sm
from statsmodels.tsa.api import stattools, AR, ARMA, VAR
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

#from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D

from keras import backend as K

# Import the data

In [2]:
data = pd.read_csv("covid_19_clean_complete.csv")

# Check the data

In [3]:
data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,Mainland China,31.8257,117.2264,1/22/20,1,0,0
1,Beijing,Mainland China,40.1824,116.4142,1/22/20,14,0,0
2,Chongqing,Mainland China,30.0572,107.8740,1/22/20,6,0,0
3,Fujian,Mainland China,26.0789,117.9874,1/22/20,1,0,0
4,Gansu,Mainland China,36.0611,103.8343,1/22/20,0,0,0


In [4]:
data.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
5817,"Placer County, CA",US,39.0916,-120.8039,3/2/20,1,0,0
5818,"San Mateo, CA",US,37.5630,-122.3255,3/2/20,1,0,0
5819,"Sarasota, FL",US,27.3364,-82.5307,3/2/20,1,0,0
5820,"Sonoma County, CA",US,38.5780,-122.9888,3/2/20,1,0,0
5821,"Umatilla, OR",US,45.7750,-118.7606,3/2/20,1,0,0


# Preprocessing

In [5]:
# Change the columns names
df = data.rename({"Province/State": "Province", "Country/Region": "Country"}, axis=1)

In [6]:
# Date time
df["Date"] = pd.to_datetime(df['Date'])

# Set time index
df = df.set_index("Date")

# Calculate death rates
df["death_rate"] = df["Deaths"] / df["Confirmed"]

,Province,Country,Lat,Long,Confirmed,Deaths,Recovered,death_rate
Date,,,,,,,,
2020-01-22,Anhui,Mainland China,31.82570,117.2264,1,0,0,0.000000
2020-01-22,Beijing,Mainland China,40.18240,116.4142,14,0,0,0.000000
2020-01-22,Chongqing,Mainland China,30.05720,107.8740,6,0,0,0.000000
2020-01-22,Fujian,Mainland China,26.07890,117.9874,1,0,0,0.000000
2020-01-22,Gansu,Mainland China,36.06110,103.8343,0,0,0,NaN
2020-01-22,Guangdong,Mainland China,23.34170,113.4244,26,0,0,0.000000
2020-01-22,Guangxi,Mainland China,23.82980,108.7881,2,0,0,0.000000
2020-01-22,Guizhou,Mainland China,26.81540,106.8748,1,0,0,0.000000
2020-01-22,Hainan,Mainland China,19.19590,109.7453,4,0,0,0.000000
